In [1]:
import cv2
import mediapipe as mp
import math

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两点之间的倾斜角度
def calculate_inclination(left_hip, right_hip):
    # 使用 atan2 计算倾斜角度
    dy = right_hip[1] - left_hip[1]  # y 的差值
    dx = right_hip[0] - left_hip[0]  # x 的差值
    angle_radians = math.atan2(dy, dx)  # 计算弧度
    angle_degrees = math.degrees(angle_radians)  # 转换为度数
    return angle_degrees

def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

# 打开视频文件
cap = cv2.VideoCapture("IMG_7239.MOV")
#cap = cv2.VideoCapture("1.png")



# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 将图像从 BGR 转换为 RGB
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)
        

        # 如果检测到骨架
        if results.pose_landmarks:
            # 获取左右髋部的坐标
            landmarks = results.pose_landmarks.landmark
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # 计算水平方向的倾斜角度
            inclination_angle = calculate_inclination(left_hip, right_hip)
            print(f"Inclination Angle between hips: {inclination_angle} degrees")

            # 绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )


        
        # 显示处理后的图像
        
        #rescaled_frame=rescale_frame(frame,percent=compress)
        cv2.imshow('Pose Estimation', frame)

        

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Inclination Angle between hips: -16.707482327299182 degrees
Inclination Angle between hips: -13.57861705229155 degrees
Inclination Angle between hips: -23.695954294709793 degrees
Inclination Angle between hips: -20.845580079289544 degrees
Inclination Angle between hips: -15.864310356864483 degrees
Inclination Angle between hips: 2.4754554185589486 degrees
Inclination Angle between hips: 9.286858434172785 degrees
Inclination Angle between hips: 14.326196029199377 degrees
Inclination Angle between hips: 13.793788411579628 degrees
Inclination Angle between hips: 9.273172519090027 degrees
Inclination Angle between hips: 7.689547025824763 degrees
Inclination Angle between hips: 10.125230174809067 degrees
Inclination Angle between hips: 12.453386863136046 degrees
Inclination Angle between hips: 12.562287153514353 degrees
Inclination Angle between hips: 14.049171900243172 degrees
Inclination Angle between hips: 13.171345151563358 degrees
Inclination Angle between hips: 12.518128521189096 degr

In [6]:
import cv2 as cv
import numpy as np

In [16]:
import cv2
import numpy as np

# 讀取背景影像（無棒球的靜態背景）
background = cv2.imread('1.png')

# 將背景轉換為灰階
gray_background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

# 讀取影片
cap = cv2.VideoCapture('IMG_7239.MOV')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 將每幀轉換為灰階
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 調整當前幀大小以匹配背景影像大小
    if gray_background.shape != gray_frame.shape:
        gray_frame = cv2.resize(gray_frame, (gray_background.shape[1], gray_background.shape[0]))

    # 計算背景與當前幀的差異
    diff = cv2.absdiff(gray_background, gray_frame)

    # 應用閾值將差異轉換為二值圖像
    _, foreground_mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)

    # 去除雜訊
    foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # 確保遮罩是 uint8 格式
    foreground_mask = foreground_mask.astype(np.uint8)

    # 檢查遮罩和圖像尺寸是否一致，並調整大小
    if foreground_mask.shape != frame.shape[:2]:
        foreground_mask = cv2.resize(foreground_mask, (frame.shape[1], frame.shape[0]))

    # 使用遮罩保留棒球部分
    foreground = cv2.bitwise_and(frame, frame, mask=foreground_mask)

    # 顯示結果
    cv2.imshow('Foreground', foreground)

    # 按 'q' 鍵退出
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()


In [18]:
import cv2
import numpy as np

# 讀取背景影像（無棒球的靜態背景）
background = cv2.imread('1.png')

# 將背景轉換為灰階
gray_background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

# 讀取影片
cap = cv2.VideoCapture('IMG_7239.MOV')


# 獲取影片的幀寬、高、和每秒幀數（fps）
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 創建VideoWriter對象來保存結果影片
output = cv2.VideoWriter('output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 如果沒有讀取到幀，則退出循環

    # 確保當前幀不為空
    if frame is None:
        print("Warning: empty frame encountered.")
        continue

    # 將每幀轉換為灰階
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 調整當前幀大小以匹配背景影像大小
    if gray_background.shape != gray_frame.shape:
        gray_frame = cv2.resize(gray_frame, (gray_background.shape[1], gray_background.shape[0]))

    # 計算背景與當前幀的差異
    diff = cv2.absdiff(gray_background, gray_frame)

    # 應用閾值將差異轉換為二值圖像
    _, foreground_mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)

    # 去除雜訊
    foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # 確保遮罩是 uint8 格式
    foreground_mask = foreground_mask.astype(np.uint8)

    # 檢查遮罩和圖像尺寸是否一致，並調整大小
    if foreground_mask.shape != frame.shape[:2]:
        foreground_mask = cv2.resize(foreground_mask, (frame.shape[1], frame.shape[0]))

    # 使用遮罩保留棒球部分
    foreground = cv2.bitwise_and(frame, frame, mask=foreground_mask)

    # 將結果寫入新影片
    output.write(foreground)

    # 顯示結果（可選）
    cv2.imshow('Foreground', foreground)

    # 按 'q' 鍵退出
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
output.release()  # 釋放影片寫入對象
cv2.destroyAllWindows()


In [17]:
import cv2
import numpy as np

# 讀取背景影像（無棒球的靜態背景）
background = cv2.imread('1.png')
if background is None:
    print("Error: Background image not found.")
    exit()

# 將背景轉換為灰階
gray_background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

# 讀取影片
cap = cv2.VideoCapture('IMG_7239.MOV')
if not cap.isOpened():
    print("Error: Video file not found or cannot be opened.")
    exit()

# 獲取影片的幀寬、高、和每秒幀數（fps）
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 創建VideoWriter對象來保存結果影片
output = cv2.VideoWriter('output_video1.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

def highlight_object(image, mask):
    """高亮顯示被檢測物體"""
    highlighted = image.copy()
    highlighted[mask > 0] = [0, 255, 255]  # 將顏色改為亮黃色
    return highlighted

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 如果沒有讀取到幀，則退出循環

    # 將每幀轉換為灰階
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 調整當前幀大小以匹配背景影像大小
    if gray_background.shape != gray_frame.shape:
        gray_frame = cv2.resize(gray_frame, (gray_background.shape[1], gray_background.shape[0]))

    # 計算背景與當前幀的差異
    diff = cv2.absdiff(gray_background, gray_frame)

    # 應用閾值將差異轉換為二值圖像
    _, foreground_mask = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)

    # 去除雜訊
    foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # 檢測輪廓
    contours, _ = cv2.findContours(foreground_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 檢查是否檢測到任何輪廓
    if contours:
        # 找到最大的輪廓
        largest_contour = max(contours, key=cv2.contourArea)

        # 獲取輪廓的邊界框
        x, y, w, h = cv2.boundingRect(largest_contour)

        # 確保掩碼與當前幀大小一致
        foreground_mask_resized = cv2.resize(foreground_mask, (frame.shape[1], frame.shape[0]))

        # 高亮顯示棒球
        highlighted_frame = highlight_object(frame, foreground_mask_resized)

        # 在原圖上繪製邊界框
        cv2.rectangle(highlighted_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 將結果寫入新影片
        output.write(highlighted_frame)

        # 顯示結果（可選）
        cv2.imshow('Highlighted Frame', highlighted_frame)

    # 按 'q' 鍵退出
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
output.release()  # 釋放影片寫入對象
cv2.destroyAllWindows()


In [17]:
import cv2
import mediapipe as mp
import math
import time

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两点之间的倾斜角度
def calculate_inclination(left_hip, right_hip):
    # 使用 atan2 计算倾斜角度
    dy = right_hip[1] - left_hip[1]  # y 的差值
    dx = right_hip[0] - left_hip[0]  # x 的差值
    angle_radians = math.atan2(dy, dx)  # 计算弧度
    angle_degrees = math.degrees(angle_radians)  # 转换为度数
    return angle_degrees

def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

# 打开视频文件
cap = cv2.VideoCapture("IMG_7239.MOV")
#cap = cv2.VideoCapture("1.png")



# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 将图像从 BGR 转换为 RGB
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)
        

        # 如果检测到骨架
        if results.pose_landmarks:
            # 获取左右髋部的坐标
            landmarks = results.pose_landmarks.landmark
            
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

            # 计算水平方向的倾斜角度
            inclination_angle = calculate_inclination(left_hip, right_hip)
            time.sleep(0.2)
            print(f"Inclination Angle between hips: {inclination_angle} degrees")

            # 绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )


        
        # 显示处理后的图像
        
        #rescaled_frame=rescale_frame(frame,percent=compress)
        cv2.imshow('Pose Estimation', frame)

        

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
        

cap.release()
cv2.destroyAllWindows()


Inclination Angle between hips: -16.707482327299182 degrees
Inclination Angle between hips: -13.57861705229155 degrees
Inclination Angle between hips: -23.695954294709793 degrees
Inclination Angle between hips: -20.845580079289544 degrees
Inclination Angle between hips: -15.864310356864483 degrees
Inclination Angle between hips: 2.4754554185589486 degrees
Inclination Angle between hips: 9.286858434172785 degrees
Inclination Angle between hips: 14.326196029199377 degrees
Inclination Angle between hips: 13.793788411579628 degrees
Inclination Angle between hips: 9.273172519090027 degrees
Inclination Angle between hips: 7.689547025824763 degrees
Inclination Angle between hips: 10.125230174809067 degrees
Inclination Angle between hips: 12.453386863136046 degrees
Inclination Angle between hips: 12.562287153514353 degrees
Inclination Angle between hips: 14.049171900243172 degrees
Inclination Angle between hips: 13.171345151563358 degrees
Inclination Angle between hips: 12.518128521189096 degr

In [2]:
import cv2
import mediapipe as mp
import math

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两个向量之间的夹角
def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]  # 向量 BA
    bc = [c[0] - b[0], c[1] - b[1]]  # 向量 BC
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cos_angle = dot_product / (magnitude_ba * magnitude_bc)
    angle = math.degrees(math.acos(cos_angle))
    return angle

# 检查角度是否在预设范围内
def check_angle(name, actual_angle, min_angle, max_angle):
    if actual_angle < min_angle:
        return f"{name} angle too low ({actual_angle:.2f} degrees). Suggested: {min_angle}-{max_angle} degrees."
    elif actual_angle > max_angle:
        return f"{name} angle too high ({actual_angle:.2f} degrees). Suggested: {min_angle}-{max_angle} degrees."
    else:
        return f"{name} angle is correct ({actual_angle:.2f} degrees)."

# 设置各关键关节的标准角度范围
standard_ranges = {
    'left_elbow': (80, 110),   # 投球过程中的肘部角度范围
    'right_elbow': (80, 110), 
    'left_shoulder': (30, 90), # 投球时肩膀角度范围
    'left_hip': (40, 80),      # 髋部的标准角度范围
    'left_knee': (30, 90)      # 膝盖的标准角度范围
}

# 打开视频文件
#cap = cv2.VideoCapture("IMG_7239.MOV")
cap = cv2.VideoCapture("1.mp4")

# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # 计算各关节角度
            left_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            )
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y], 
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y], 
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )
            left_shoulder_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            )
            left_hip_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            )
            left_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y], 
                [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            )

            # 检查角度与标准范围的差距并提供建议
            feedback_left_elbow = check_angle('Left Elbow', left_elbow_angle, *standard_ranges['left_elbow'])
            feedback_right_elbow = check_angle('Right Elbow', right_elbow_angle, *standard_ranges['right_elbow'])
            
            feedback_shoulder = check_angle('Left Shoulder', left_shoulder_angle, *standard_ranges['left_shoulder'])
            feedback_hip = check_angle('Left Hip', left_hip_angle, *standard_ranges['left_hip'])
            feedback_knee = check_angle('Left Knee', left_knee_angle, *standard_ranges['left_knee'])

          
            # 在图像上绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

        
        # 显示处理后的图像
        cv2.imshow('Pitcher Pose Analysis', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
if check_angle('Right Elbow', right_elbow_angle, *standard_ranges['right_elbow']):
            print(feedback_right_elbow)
            print("手肘角度需調整")
cap.release()
cv2.destroyAllWindows()


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Right Elbow angle too high (175.19 degrees). Suggested: 80-110 degrees.
手肘角度需調整


In [6]:
import cv2
import mediapipe as mp
import math

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两个向量之间的夹角
def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]  # 向量 BA
    bc = [c[0] - b[0], c[1] - b[1]]  # 向量 BC
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cos_angle = dot_product / (magnitude_ba * magnitude_bc)
    angle = math.degrees(math.acos(cos_angle))
    return angle

# 检查右手肘是否高于肩膀
def is_right_elbow_above_shoulder(landmarks):
    return landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y < landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y

# 检查右手肘角度
def check_right_elbow_angle(angle):
    return 160 <= angle <= 170

# 打开视频文件
cap = cv2.VideoCapture("IMG_7239.MOV")  # 请替换为您的视频文件路径

correct_angle_detected = False

# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    detection_started = False
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 将图像从 BGR 转换为 RGB
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # 计算右手肘角度
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )

            # 检查是否开始检测
            if not detection_started and is_right_elbow_above_shoulder(landmarks):
                detection_started = True

            # 如果已开始检测，则进行角度检查
            if detection_started:
                if check_right_elbow_angle(right_elbow_angle):
                    correct_angle_detected = True

                # 如果达到结束条件，重置检测状态
                if right_elbow_angle > 170:
                    detection_started = False

            # 绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

        # 显示处理后的图像
        cv2.imshow('Pitcher Pose Analysis', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 输出结果
if correct_angle_detected:
    print("在投球过程中，右手肘角度有达到正确范围（160-170度）。")
else:
    print("在整个投球过程中，右手肘角度未达到正确范围（160-170度）。")

在投球过程中，右手肘角度有达到正确范围（160-170度）。


In [10]:
import cv2
import mediapipe as mp
import math
import numpy as np

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两个向量之间的夹角
def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]  # 向量 BA
    bc = [c[0] - b[0], c[1] - b[1]]  # 向量 BC
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cos_angle = dot_product / (magnitude_ba * magnitude_bc)
    angle = math.degrees(math.acos(max(-1, min(1, cos_angle))))
    return angle

# 检查右手肘是否高于肩膀
def is_right_elbow_above_shoulder(landmarks):
    return landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y < landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y

# 检查右手肘角度
def check_right_elbow_angle(angle):
    return 160 <= angle <= 170

# 检查左膝盖角度
def check_left_knee_angle(angle):
    return angle >= 170  # 假设大于170度视为"接近伸直"

# 检查脚尖朝向
def check_foot_direction(landmarks):
    left_foot = np.array([landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y])
    right_foot = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y])
    foot_direction = left_foot - right_foot
    # 假设y轴正方向为捕手方向，检查脚尖是否大致朝向y轴正方向
    return foot_direction[1] > 0

# 打开视频文件
cap = cv2.VideoCapture("IMG_7239.MOV")  # 请替换为您的视频文件路径

correct_elbow_angle_detected = False
correct_knee_angle_detected = False
correct_foot_direction_detected = False

# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    detection_started = False
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 将图像从 BGR 转换为 RGB
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # 计算右手肘角度
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )

            # 计算左膝盖角度
            left_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            )

            # 检查是否开始检测
            if not detection_started and is_right_elbow_above_shoulder(landmarks):
                detection_started = True

            # 如果已开始检测，则进行各项检查
            if detection_started:
                if check_right_elbow_angle(right_elbow_angle):
                    correct_elbow_angle_detected = True
                if check_left_knee_angle(left_knee_angle):
                    correct_knee_angle_detected = True
                if check_foot_direction(landmarks):
                    correct_foot_direction_detected = True

                # 如果达到结束条件，重置检测状态
                if right_elbow_angle > 170:
                    detection_started = False

            # 绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

        # 显示处理后的图像
        cv2.imshow('Pitcher Pose Analysis', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 输出结果
print("投球姿势分析结果：")
if correct_elbow_angle_detected:
    print("- 右手肘角度有达到正确范围（160-170度）。")
else:
    print("- 右手肘角度未达到正确范围（160-170度）。")

if correct_knee_angle_detected:
    print("- 投球动作结束时，左膝盖接近伸直。")
else:
    print("- 投球动作结束时，左膝盖未充分伸直。")

if correct_foot_direction_detected:
    print("- 脚尖正确朝向捕手方向。")
else:
    print("- 脚尖未正确朝向捕手方向。")

投球姿势分析结果：
- 右手肘角度有达到正确范围（160-170度）。
- 投球动作结束时，左膝盖接近伸直。
- 脚尖正确朝向捕手方向。


In [3]:
import cv2
import mediapipe as mp
import math
import numpy as np

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两个向量之间的夹角
def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]  # 向量 BA
    bc = [c[0] - b[0], c[1] - b[1]]  # 向量 BC
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cos_angle = dot_product / (magnitude_ba * magnitude_bc)
    angle = math.degrees(math.acos(max(-1, min(1, cos_angle))))
    return angle

# 检查右手肘是否高于肩膀
def is_right_elbow_above_shoulder(landmarks):
    return landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y < landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y

# 检查右手肘角度
def check_right_elbow_angle(angle):
    return 160 <= angle <= 170

# 打开视频文件
cap = cv2.VideoCapture("IMG_7239.MOV")  # 请替换为您的视频文件路径

correct_elbow_angle_detected = False
detection_phase = "Not started"  # 用于追踪检测阶段

cv2.namedWindow('Pitcher Pose Analysis', cv2.WINDOW_NORMAL)
# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        
        # 将图像从 BGR 转换为 RGB
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # 计算右手肘角度
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )

            # 检测逻辑
            if detection_phase == "Not started" and is_right_elbow_above_shoulder(landmarks):
                detection_phase = "Started"
                print("检测开始：右手肘高于肩膀")
            
            elif detection_phase == "Started":
                if check_right_elbow_angle(right_elbow_angle):
                    correct_elbow_angle_detected = True
                
                if is_right_elbow_above_shoulder(landmarks) and right_elbow_angle > 170:
                    detection_phase = "Ended"
                    print("检测结束：右手肘高于肩膀且角度大于170度")

            # 绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

        # 显示处理后的图像
        cv2.imshow('Pitcher Pose Analysis', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 输出结果
print("\n投球姿势分析结果：")
if correct_elbow_angle_detected:
    print("在检测过程中，右手肘角度有达到正确范围（160-170度）。")
else:
    print("在整个检测过程中，右手肘角度未达到正确范围（160-170度）。")

检测开始：右手肘高于肩膀

投球姿势分析结果：
在检测过程中，右手肘角度有达到正确范围（160-170度）。


In [4]:
import sys
import io
sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

import cv2
import mediapipe as mp
import math
import numpy as np

# 初始化 MediaPipe Pose 模型
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 计算两个向量之间的夹角
def calculate_angle(a, b, c):
    ba = [a[0] - b[0], a[1] - b[1]]  # 向量 BA
    bc = [c[0] - b[0], c[1] - b[1]]  # 向量 BC
    dot_product = ba[0] * bc[0] + ba[1] * bc[1]
    magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cos_angle = dot_product / (magnitude_ba * magnitude_bc)
    angle = math.degrees(math.acos(max(-1, min(1, cos_angle))))
    return angle

# 检查右手肘是否高于肩膀
def is_right_elbow_above_shoulder(landmarks):
    return landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y < landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y

# 检查右手肘角度
def check_right_elbow_angle(angle):
    return 160 <= angle <= 170

# 检查左膝盖角度
def check_left_knee_angle(angle):
    return angle >= 170  # 假设大于170度视为"接近伸直"

# 检查脚尖朝向
def check_foot_direction(landmarks):
    left_foot = np.array([landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y])
    right_foot = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y])
    foot_direction = left_foot - right_foot
    # 假设y轴正方向为捕手方向，检查脚尖是否大致朝向y轴正方向
    return foot_direction[1] > 0

# 打开视频文件
cap = cv2.VideoCapture("IMG_7239.MOV")  # 请替换为您的视频文件路径

correct_elbow_angle_detected = False
correct_knee_angle_detected = False
correct_foot_direction_detected = False
detection_phase = "Not started"  # 用于追踪检测阶段

cv2.namedWindow('Pitcher Pose Analysis', cv2.WINDOW_NORMAL)
# 使用 MediaPipe Pose 进行骨架检测
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 将图像从 BGR 转换为 RGB
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(img_rgb)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # 计算右手肘角度
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )

            # 计算左膝盖角度
            left_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            )

            # 检测逻辑
            if detection_phase == "Not started" and is_right_elbow_above_shoulder(landmarks):
                detection_phase = "Started"
                print("检测开始：右手肘高于肩膀")
            
            elif detection_phase == "Started":
                if check_right_elbow_angle(right_elbow_angle):
                    correct_elbow_angle_detected = True
                
                if check_left_knee_angle(left_knee_angle):
                    correct_knee_angle_detected = True
                
                if check_foot_direction(landmarks):
                    correct_foot_direction_detected = True
                
                if is_right_elbow_above_shoulder(landmarks) and right_elbow_angle > 170:
                    detection_phase = "Ended"
                    print("检测结束：右手肘高于肩膀且角度大于170度")

            # 绘制骨架
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

        # 显示处理后的图像
        cv2.imshow('Pitcher Pose Analysis', frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 输出结果
print("\n投球姿势分析结果：")
if correct_elbow_angle_detected:
    print("- 在检测过程中，右手肘角度有达到正确范围（160-170度）。")
else:
    print("- 在整个检测过程中，右手肘角度未达到正确范围（160-170度）。")

if correct_knee_angle_detected:
    print("- 投球动作结束时，左膝盖接近伸直。")
else:
    print("- 投球动作结束时，左膝盖未充分伸直。")

if correct_foot_direction_detected:
    print("- 脚尖正确朝向捕手方向。")
else:
    print("- 脚尖未正确朝向捕手方向。")

AttributeError: 'OutStream' object has no attribute 'buffer'